### Reading dataset in

In [15]:
import pandas as pd
df = pd.read_csv("daft_ie_v1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3967 entries, 0 to 3966
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3967 non-null   int64  
 1   title             3967 non-null   object 
 2   featuredLevel     3967 non-null   object 
 3   publishDate       3967 non-null   object 
 4   price             3967 non-null   int64  
 5   numBedrooms       3967 non-null   int64  
 6   numBathrooms      3967 non-null   int64  
 7   propertyType      3967 non-null   object 
 8   propertySize      3612 non-null   float64
 9   category          3967 non-null   object 
 10  AMV_price         3967 non-null   int64  
 11  sellerId          3967 non-null   float64
 12  seller_name       3967 non-null   object 
 13  seller_branch     3967 non-null   object 
 14  sellerType        3967 non-null   object 
 15  m_totalImages     3967 non-null   float64
 16  m_hasVideo        3967 non-null   bool   


### Cleaning

In [36]:
import plotly.express as px
#scatter of longitude and altitude showed a value that was definitely an outlier
px.scatter(df, x='longitude', y='latitude', trendline='ols',
           title='longitude vs latitude').show()

# this value lines up wiht somewhere in america so we will remove
df = df[~df['latitude'].isin([39.78373])]

In [3]:
df.propertyType.value_counts()
# removing low count values that would interfer with our model
df = df[~df['propertyType'].isin(['Townhouse', 'Duplex','Site','House','Studio'])]
df.propertyType.value_counts()
print(len(df))

3832


### Histograms
##### log-transforming

What we're looking for
- Original scale: if the histogram is strongly right‑skewed, a log transform often helps.
- Log scale: if the histogram of log_price looks more symmetric, bell‑ish, that’s a sign that log‑transforming for regression is useful (errors closer to normal, linearity easier).

In [37]:
import numpy as np

# price and size log-transform)
df['log_price'] = np.log1p(df['price'])          # log(1 + x) helps to avoid log(0)
df['log_size']  = np.log1p(df['propertySize'])

# histograms before transformation
px.histogram(df, x='price', nbins=50, title='Price').show()
px.histogram(df, x='log_price', nbins=50, title='log(price)').show()

#log- scale histograms
px.histogram(df, x='propertySize', nbins=50, title='Property size').show()
px.histogram(df, x='log_size', nbins=50, title='log(size)').show()

### Splitting address to get town and county
Title would be useful as it contains information on location, rather than just longitude and latitude to access the county and other location information we need to split the title up

In [8]:
#looking at pattern in title column
# print(df['title'].value_counts())

# regex pattern to get Co. xxx and Dublin 15 etc.
county_pattern = r'(Co\.\s+\w+|Dublin\s+\d+)$'
df['county'] = df['title'].str.extract(county_pattern)

# collecting the rest of the address (less the county)
df['no_county'] = df['title'].str.replace(
    r',?\s*(Co\.\s+\w+|Dublin\s+\d+)$', 
    '', 
    regex=True
)

# Without county, town will be next comma, leaving address (estate or house 1 or 2 lines) to left
tmp = df['no_county'].str.rsplit(',', n=1, expand=True)
df['address_part'] = tmp[0].str.strip()
df['town'] = tmp[1].str.strip()

# drop helper column
df = df.drop(columns=['no_county'])
print(len(df.columns))

25


### EDA & Preprocessing

In [18]:
import plotly.express as px

# define which columns would be useful to know about outliers
# This could be updated and rerun after any changes were made
outlier_cols = ['price', 'numBedrooms', 'numBathrooms', 'propertySize']

for col in outlier_cols:
    fig = px.box(df, y=col, points='suspectedoutliers', title=f"{col} - Boxplot")
    fig.show()

In [ ]:
### noticing outliers in boxplots and investiating further
df.loc[df['price'].isin([4500000])] # Not an outlier but will it distort our model?
df.loc[df['numBedrooms'].isin([23])] # Can not find it existing online, may be an outlier
df.loc[df['propertySize'].isin([8600, 8094])] # Not an outlier but will it distort our model?

### IQR

In [33]:
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    IQR = Q3 - Q1

    # define lower bound and upper bound
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DF to keep outliers (remove only between lower_bound and upper_bound)
    no_outliers_df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return no_outliers_df

# PRICE
# Define function to remove outliers

no_outliers_df = remove_outliers(df, 'price')
no_outliers_df = remove_outliers(df, 'numBedrooms')
no_outliers_df = remove_outliers(df, 'numBathrooms')
no_outliers_df = remove_outliers(df, 'propertySize')


for col in outlier_cols:
    fig = px.box(no_outliers_df, y=col, points='suspectedoutliers', title=f"{col} - Boxplot")
    fig.show()

In [34]:
# From boxplot above notice very small propertySize squared
# propertySize msut be at least 20m^2
no_outliers_df = no_outliers_df[no_outliers_df['propertySize'] >= 20]

### Scatterplots
##### Relationships with price
- price vs propertySize
    - raw scale, dominated by outliers
    - on log-log relationship is much clearer and roughly linear, use log_price as target and log_size as one of predictors

- bedrooms and bathrooms
    - both have an upward relationship with price 

In [39]:
# price vs propertySize
px.scatter(no_outliers_df, x='propertySize', y='price', trendline='ols',
           title='Price vs Property size').show()

# log transformations on price and size same as histograms
no_outliers_df['log_price'] = np.log1p(no_outliers_df['price'])          
no_outliers_df['log_size']  = np.log1p(no_outliers_df['propertySize'])
px.scatter(no_outliers_df, x='log_size', y='log_price', trendline='ols',
           title='log Price vs log Property size').show()

# price vs numBedrooms
px.scatter(no_outliers_df, x='numBedrooms', y='price', trendline='ols',
           title='Price vs Bedrooms').show()

# price vs numBathrooms
px.scatter(no_outliers_df, x='numBathrooms', y='price', trendline='ols',
           title='Price vs Bathrooms').show()

# price vs numBathrooms
px.scatter(no_outliers_df, x='numBathrooms', y='numBedrooms', trendline='ols',
           title='Bedrooms vs Bathrooms').show()

# Simple linear regression model
Utilising ordinary least squares model from statsmodels library
- Using our no_outliers_df we train a baseline multiple linear regression model including all numeric features, simple to get some baseline results to improve on

In [43]:
import statsmodels.api as sm
import numpy as np

no_outliers_df = no_outliers_df.dropna()

features = ['title', 'featuredLevel', 'publishDate', 'numBedrooms',
 'numBathrooms', 'propertyType', 'propertySize', 'category', 'AMV_price',
 'sellerId', 'seller_name', 'seller_branch', 'sellerType', 'm_totalImages',
 'm_hasVideo', 'm_hasVirtualTour', 'm_hasBrochure', 'ber_rating', 'longitude',
 'latitude']

# include all numeric features
x = no_outliers_df[features].select_dtypes(include=[np.number])

y = no_outliers_df['price']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     91.25
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          1.71e-137
Time:                        13:30:14   Log-Likelihood:                -45621.
No. Observations:                3364   AIC:                         9.126e+04
Df Residuals:                    3355   BIC:                         9.132e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1.665e+05   2.79e+05      0.596

### Linear Regression model with more specific features
- Limiting the number of features in our simple model should improve performance
- For this example we want to only select propertySize

In [52]:
import statsmodels.api as sm
import numpy as np


features = ['numBedrooms', 'numBathrooms', 'propertyType', 'propertySize', 'longitude', 'latitude']

# include selected features
x = no_outliers_df[features].select_dtypes(include=[np.number])

y = no_outliers_df['price']

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     125.2
Date:                Fri, 28 Nov 2025   Prob (F-statistic):          7.32e-122
Time:                        13:35:05   Log-Likelihood:                -45665.
No. Observations:                3364   AIC:                         9.134e+04
Df Residuals:                    3358   BIC:                         9.138e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         4.326e+04   2.82e+05      0.154   